In [1]:
# !pip install httpx pandas numpy beautifulsoup4
# !pip install -e .
%load_ext autoreload
%autoreload 2

import asyncio
import httpx
import urllib
import datetime
import numpy as np
import pandas as pd
import dataclasses
import typing
import math
import time
import calendar
from bs4 import BeautifulSoup

import nest_asyncio
nest_asyncio.apply()

from macro_scrape.headers import *

In [2]:
import asyncio
import json
from typing import Optional, List, Literal, Union


@dataclasses.dataclass
class SecuritySearchObj:
    isinType            : Literal['Primary Isin', ]     = 'Primary Isin'
    instrumentCategory  : Literal['01-All']             = '01-All'
    annaStatus          : Literal['1 All']              = '1 All'
    country             : Literal['All',]               = 'All'
    issueCurrency       : Literal['All',]               = 'All'
    entityNameOpton1    : Literal['1-ISSUER_NAMELONG',] = '1-ISSUER_NAMELONG'
    entityNameOpton2    : Literal['3 Contains',]        = '3 Contains'
    Search              : Literal['Search']             = 'Search'
    entityName          : Optional[str] = None
    isinValue           : Optional[str] = None
    rateFrom            : Optional[str] = None
    rateTo              : Optional[str] = None
    maturityFrom        : Optional[str] = None
    maturityTo          : Optional[str] = None
    updateFrom          : Optional[str] = None
    updateTo            : Optional[str] = None



base_url = 'https://www.annaservice.com'


@dataclasses.dataclass
class ISINSearchObj:
    '''Logs into Anna ISIN search service
    '''
    username    : str
    password    : str

    timeout     : float                         = 10.0
    client      : Optional[httpx.AsyncClient]   = None
    _csrf       : Optional[str]                 = None


    async def __aenter__(self,):
        self.client = await httpx.AsyncClient(headers=headers, timeout=self.timeout).__aenter__()
        await self.login()
        return self

    async def __aexit__(self, *args, **kwargs):
        await self.client.__aexit__(*args, **kwargs)


    @staticmethod
    def get_page_secret(text:str) -> str:
        soup = BeautifulSoup(text)
        id_tags = soup.findAll("input", {"type" : "hidden", 'name':'_csrf'})
        id_tag = id_tags[0]
        secret = id_tag['value']
        return secret


    async def login(self,):
        landing_url = urllib.parse.urljoin(base_url,'/isinlookup/login')
        login_url = urllib.parse.urljoin(base_url,'/isinlookup/authenticate')
        page = await self.client.get(landing_url)
        page.raise_for_status()
        login_data = {
            'username':self.username,
            'password':self.password,
            '_csrf': ISINSearchObj.get_page_secret(page.text),
        }

        page = await self.client.post(login_url, data=login_data, follow_redirects=True)
        page.raise_for_status()
        # print('logged in successfully')
        # Give the client the secret we were given
        self._csrf = ISINSearchObj.get_page_secret(page.text)
    

    async def search_securities(self, search_obj : SecuritySearchObj):
        search_url = urllib.parse.urljoin(base_url,'/isinlookup/search')

        data = dataclasses.asdict(search_obj)

        if self._csrf is None:
            page = await self.client.get(search_url, follow_redirects=True)
            page.raise_for_status()
            self._csrf = ISINSearchObj.get_page_secret(page.text)
        
        data['_csrf'] = self._csrf

        page = await self.client.post(search_url, data = data, follow_redirects=True)
        self._csrf = None
        page.raise_for_status()
        # print(page.text)
        response = page.text.split('eval("results = " +')[1].split(';')[0][1:-2]
        # print(response)
        response_js = json.loads(response)
        return pd.DataFrame(response_js['response']['isinRecords'])
        # return None


In [3]:
df = None
    

async def main():
    global df
    async with ISINSearchObj(username=username, password=password) as client:
        obj = SecuritySearchObj(
            isinValue = 'US0378331005',
            # entityName = 'apple',
        )
        df = await client.search_securities(search_obj = obj)
        print(df)


asyncio.run(main())


NameError: name 'username' is not defined

In [ ]:

df.iloc[0]


fisn                  APPLE INC/SH SH
instrumentCategory                 EQ
issueCurrency                     USD
issuerNameLong              APPLE INC
entryDate                  06/26/2001
modifiedDate               11/25/2021
cfiCode                        ESVUFR
isin                     US0378331005
issueDescription                  COM
issuerRegCountry                   US
status                              N
Name: 0, dtype: object

In [18]:
dt = datetime.date(2022,12,9)
# uri = 'ftp://ftp.cmegroup.com/irs/CME_ATM_VolCube_{0:%Y%m%d}.csv'.format(dt)
# print(uri)

# with CMEIRSResource( uri = uri, ) as client:

#     print(client.last_modified().astimezone(pytz.timezone('US/Eastern')))

#     if not client.local_copy_exists():
#         print('downloading file')
#         client.download_file()

#     df = client.load_from_local()

# print(df.head())
# # print(df.iloc[[0]].T)
import macro_scrape.cme.irs
df = macro_scrape.cme.irs.CMEIRSResource.get_atm_vol_on_date(dt)

local file found
NoneType: None


In [19]:
df

,Currency,Expiry,UnderlyingTenor,Moneyness,Strike,NormalVol,LogNormalVol,OptionPrice,Annuity
Date,,,,,,,,,
2022-12-09,USD,1M,10Y,0,0.035480,0.013502,0.380735,0.013120,8.357944
2022-12-09,USD,1Y,10Y,0,0.033649,0.012470,0.372738,0.040511,8.121116
2022-12-09,USD,2Y,10Y,0,0.032847,0.011671,0.359154,0.051939,7.882222
2022-12-09,USD,3M,10Y,0,0.035187,0.013337,0.379583,0.021966,8.314203
2022-12-09,USD,6M,10Y,0,0.034671,0.013076,0.378260,0.030378,8.246947
2022-12-09,USD,1M,15Y,0,0.035309,0.012846,0.363991,0.017424,11.666197
2022-12-09,USD,1Y,15Y,0,0.033964,0.011709,0.346484,0.053087,11.333505
2022-12-09,USD,2Y,15Y,0,0.033278,0.010937,0.331672,0.067903,10.996830
2022-12-09,USD,3M,15Y,0,0.035099,0.012666,0.361340,0.029120,11.605765


In [48]:
fldr = 'ftp://ftp.cmegroup.com/irs/' #CME_ATM_VolCube_{0:%Y%m%d}.csv'.format(dt)
# print(uri)

fnames = []
with CMEIRSResource( uri = fldr, ) as client:
    # print(dir(client))
    client.login()
    # f = lambda line: fnames.append( line[29:].strip().split(' ')[4:] )
    # client.ftp.dir(fnames.append)
    res = client.ftp.retrlines('LIST')

# print(len('drwxr-xr-x    3 ftp      ftp          6144 Jul 15 16:00  '))
# fnames
# res

drwxr-xr-x    3 ftp      ftp          6144 Jul 15 16:30  span
drwxr-xr-x    7 ftp      ftp          6144 Jul 15 16:30 SBEFix
drwxr-xr-x    2 ftp      ftp          6144 Dec 09 17:01 TCF
drwxr-xr-x    2 ftp      ftp         22528 Dec 05 01:20 bitcoinfutures
drwxr-xr-x    2 ftp      ftp         26624 Dec 05 09:20 btic
drwxr-xr-x    2 ftp      ftp         14336 Dec 09 21:40 bulletin
drwxr-xr-x    5 ftp      ftp          6144 Jul 15 16:32 cash_settled_commodity_index_prices
drwxr-xr-x    2 ftp      ftp         79872 Dec 10 10:51 cash_settled_grain_contracts
drwxr-xr-x    2 ftp      ftp        112640 Dec 09 04:31 daily_volume
drwxr-xr-x    6 ftp      ftp          6144 Jul 15 16:32 datamine_sample_data
drwxr-xr-x    6 ftp      ftp          6144 Dec 05 14:58 delivery_reports
drwxr-xr-x    2 ftp      ftp         34816 Dec 05 09:21 dividendfutures
-rw-r--r--    1 ftp      ftp             0 May 15  2018 east-aws-ftp-webwatch-monitor-donotdelete
drwxr-xr-x    3 ftp      ftp         38912 Jul 15 16

In [38]:
import urllib3
import os

os.path.dirname( urllib3.util.parse_url(uri).path )


'/irs'